# MiniMax Algorithm: Forming Game Tree
With this algorithm: You are trying to **maximize** your score, while your opponent is trying to **minimize** your score. Assuming your opponent is as smart as you. 

#### Equation:

### $$\overline{u_i} = min_{s_-i}max_{s_i}u_i(s_i, s_-i)$$

**Function A:** A function that finds a move that immediately wins the game
1. Loop over all **legal moves**
2. Calculate what the board would look like if you pick this move
3. IF this is the winning move, no need to continue searching & pick that move
4. Else can't win on this move (will loop through this next move)

In [3]:
def find_winning_move(game_state, next_player):
    
    # Loop over all **legal moves**
    for candidate_move in game_state.legal_moves(next_player): 
        
        # Calculate what the board would look like if you pick this move
        next_state = game_state.apply_move(candidate_move) 
        
        # IF this is the winning move, no need to continue searching & pick that move
        if next_state.is_over() and next_state.winner == next_player: 
            
            return cadidate_move 
    # Else can't win on this move (will loop through this next move)
    return None  

**Function B:** A function that avoids giving the opponent a winning move
1. ```possible_moves``` becomes a **list** of all moves worth considering
2. Loops over all **legal moves**
3. Calculate what the board would look like if you play this move
4. Does this give your **opponent** a **winning move**?
5. IF NOT, this move is plausible

In [4]:
def eliminate_losing_moves(game_state, next_player):
    opponent = next_player.other()
    possible_moves = []
    
    # We will be looping through all legal moves
    for candidate_move in game_state.legal_moves(next_player):
        
        # Grabbing the next game state with that legal move
        next_state = game_state.apply_move(candidate_moves)
        
        # Calling our find_winning_move() function to see if that state could result in a winning move
        opponent_winning_move = find_winning_move(next_state, opponent)
        
        # If it doesn't we add this candidate move
        if opponent_winning_move is None:
            possible_moves.append(candidate_move)
            
    # Returning all possible winning moves         
    return possble_moves

**Function C:** A function that finds a **two-move** sequence that guarentees a win
1. Loops over all **legal moves**
2. Calculate what the board would look like if you play this move
3. Does your **opponent** have **good defense?*
4. IF NOT, Pick this move

In [5]:
def find_two_step_win(game_state, next_player):
    opponent = next_player.other()
    
    # Looping through all legal moves
    for candidate_move in game_state.legal_moves(next_player):
        
        # For all legal moves, what is the next game state?
        next_state = game_state.apply_move(candidate_move)
        
        # Does the opponent have any good moves given the game state?
        good_responses = eliminate_losing_moves(next_state, opponent)
        
        # IF not then that is our move
        if not good_responses:
            return candidate_moves
        
    # otherwise, you're screwed buddy
    return None

### Tic-Tac-Toe Example
In this following example we will simple play tic-tac-toe

In [20]:
# Our imports
from enum import Enum
import random

In [21]:
# Declaring Classes
class GameResult(Enum):
    """
    Using Enum to represent the outcome of the game
    """
    loss = 1
    draw = 2
    win = 3
    
class MinimaxAgent():
    """
    A Game playing agent that impliments MiniMax Search
    
    when using this function: change this to: 
        1. Create Agent Class
        2. MinimaxAgent(Agent) - inherets from Agent class
    """
    def select_move(self, game_state):
        winning_moves = []
        draw_moves = []
        losing_moves = []
        
        # Let's loop through all legal moves
        for possible_move in game_state.legal_moves():
            next_state = game_state.apply_move(possible_move)
            
            # Grabbing our opponents best outcome
            opponent_best_outcome = best_result(next_state)
            
            # Grabbing our best outcome from the opponents best outcome
            our_best_outcome = reverse_game_result(opponent_best_outcome)
            
            # Does our best outcome result in a win?
            if our_best_outcome == GameResult.win:    
                winning_moves.append(possible_move)
            
            # Does our best outcome result in a draw?
            elif our_best_outcome == GameResult.draw:
                draw_moves.append(possible_move)
            
            # Otherwise it results in a loss
            else:
                losing_moves.append(possible_move)
        
        # Now we pick a move that leads to best outcome
        if winning_moves:
            return random.choice(winning_moves)
        
        if draw_moves:
            return random.choice(draw_moves)
        
        return random.choice(losing_moves)

In [ ]:
# Game functions
def best_result(game_state):
    """
    The first step of the minimax algorithm. 
    We start from the end of the game and work backward. If the game is already over, there is only one possible result.
    Returns: GameResult.win | GameResult.draw | GameResult.loss
    """
    # Is the game over?
    if game_state.is_over():
        
        # Is this player the winner of the game? - in our class this will be the opponent
        if game_state.winner() == game_state.next_player:
            return GameResult.win
        
        # Is there no winner?
        elif game_state.winner() is None:
            return GameResult.draw

        # Otherwise
        else:
            return GameResult.loss
    
    # Initializing at lowest possible result = 1
    best_result_so_far = GameResult.loss
    
    # Looping through all legal moves
    for candidate_move in game_state.legal_moves():
        
        # Grabbing next game state for that candidate move
        next_state = game_state.apply_move(candidate_move)
        
        # Calculating our opponents best result given that game state
        opponent_best_result = best_result(next_state)
        
        # Calculating our result given if they play their best result
        our_result = reverse_game_result(opponent_best_result)
        
        # win = 3, draw = 2, loss = 1: Let's grab the best result if it exist
        if our_result.value > best_result_so_far.value:
            best_result_so_far = our_result
        
    # Returning that best result, 1 if we don't have a superior move
    return best_result_so_far
        

def reverse_game_result(opponents_game_result):
    """
    This function will take opponents_best_outcome and determine the best counter measure
    Returns:
        our_best_outcome
    """
    # Does their game result end in a loss given that state?
    if opponents_game_result == GameResult.loss:
        return GameResult.win # we win!
    
    # Does their game result end in a win given that state?
    if opponenets_game_result == GameResult.win:
        return GameResult.loss # we lost...
    
    # otherwise
    return GameResult.draw